In [3]:
%pip install torchviz

import torch
import torch.nn as nn
from torchviz import make_dot


  Preparing metadata (setup.py) ... done
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4150 sha256=02d8e616bda3de9755f85a962f23cfbc7e081429eee0865900f39af8bb3d5b1a
  Stored in directory: /Users/psaghelyi-remote/Library/Caches/pip/wheels/29/65/6e/db2515eb1dc760fecd36b40d54df65c1e18534013f1c037e2e
Successfully built torchviz

[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:

# Define the custom model as per the diagram provided
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        
        # Define the first part of the CNN
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=128, kernel_size=3, stride=2, padding='valid')
        self.conv2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding='same')
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=1, stride=1, padding='same')
        
        # Define the second part of the CNN
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding='same')
        self.conv5 = nn.Conv2d(in_channels=64, out_channels=256, kernel_size=5, stride=4, padding=1)
        
        # Define the batch normalization layer
        self.batch_norm = nn.BatchNorm2d(num_features=256)
        
        # Define the third part of the CNN
        self.conv6 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=5, stride=1, padding='same')
        
        # Define the LSTM layer
        self.lstm = nn.LSTM(input_size=256, hidden_size=128, batch_first=True)
        
        # Define the linear layer
        self.fc = nn.Linear(in_features=128, out_features=13)
        
        # Define the sigmoid activation
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Forward pass through the first part of the CNN
        x1 = self.conv1(x)
        x2 = self.conv2(x1)
        x3 = self.conv3(x2)
        
        # Forward pass through the second part of the CNN
        x4 = self.conv4(x3)
        x5 = self.conv5(x4)
        
        # Channel-wise concatenation
        x_concat = torch.cat((x3, x5), 1)
        
        # Forward pass through the batch normalization
        x_bn = self.batch_norm(x_concat)
        
        # Forward pass through the third part of the CNN
        x6 = self.conv6(x_bn)
        
        # Residual connection and reshaping for LSTM
        x_residual = x6 + x_bn
        x_reshape = x_residual.view(x_residual.size(0), -1, x_residual.size(1))
        
        # Forward pass through LSTM
        lstm_out, (hn, cn) = self.lstm(x_reshape)
        
        # Select the last time-step from LSTM's output
        lstm_last_out = lstm_out[:, -1, :]
        
        # Forward pass through the linear layer
        x_fc = self.fc(lstm_last_out)
        
        # Forward pass through the sigmoid activation
        out = self.sigmoid(x_fc)
        
        return out

# Create the model instance
model = CustomModel()

# Display the model architecture
model



# Please note that we haven't compiled the model or prepared it for training since the prompt only asked for implementation.


CustomModel(
  (conv1): Conv2d(3, 128, kernel_size=(3, 3), stride=(2, 2), padding=valid)
  (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (conv3): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), padding=same)
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (conv5): Conv2d(64, 256, kernel_size=(5, 5), stride=(4, 4), padding=(1, 1))
  (batch_norm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv6): Conv2d(256, 256, kernel_size=(5, 5), stride=(1, 1), padding=same)
  (lstm): LSTM(256, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=13, bias=True)
  (sigmoid): Sigmoid()
)

In [6]:
x = torch.randn(1, 3, 224, 224).requires_grad_(True)
y = model(x)

make_dot(y, params=dict(list(model.named_parameters()) + [('input', x)]))

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 111 but got size 28 for tensor number 1 in the list.

In [24]:
#tensor = torch.randn(1, 2, 3)
tensor = torch.randn(1, 2, 3).requires_grad_(True)
print(tensor)

tensor([[[ 0.4612,  0.2420, -0.1722],
         [ 0.2322, -0.5171,  0.6477]]], requires_grad=True)


In [21]:
x
#x.unsqueeze(0)

tensor([[1, 2, 3],
        [4, 5, 6]])